In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# 모델과 토크나이저 로드
model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
# 프롬프트와 인스트럭션 정의
PROMPT = '''당신은 고성능 한국어 대화 요약 AI입니다. 주어진 대화를 읽고 아래의 지침에 따라 핵심 내용을 간결하게 요약해서 요약문만 반환해주세요.'''
instruction = '''
1. 대화에서 가장 중요한 정보를 파악하고 전달하세요.
2. 요약은 원본 대화 길이의 20% 이내로 간략하게 작성하세요.
3. 대화 내에서 언급된 중요한 명명된 개체(예: 사람 이름, 기업명 등)를 보존하세요.
4. 관찰자의 관점에서 요약을 작성하되, 화자들의 의도를 이해하고 반영하세요.
5. 은어나 약어를 사용하지 말고, 공식적으로 사용되는 언어로 작성하세요.
6. 대화의 전체적인 맥락과 주요 주제를 파악하세요.
7. 중요한 정보와 결론을 추출하여 논리적으로 구성하세요.
---
#Person1#: 안녕하세요, 오늘 하루 어떠셨어요? 
#Person2#: 요즘 숨쉬기가 좀 힘들어요.
#Person1#: 최근에 감기 같은 것에 걸리신 적이 있나요?
#Person2#: 아니요, 감기는 아니에요. 그냥 숨을 쉴 때마다 가슴이 무겁게 느껴져요.
#Person1#: 알고 있는 알레르기가 있나요?
#Person2#: 아니요, 알고 있는 알레르기는 없어요.
#Person1#: 이런 증상이 항상 나타나나요, 아니면 활동할 때 주로 나타나나요?
#Person2#: 운동을 할 때 많이 나타나요.
#Person1#: 저는 당신을 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 거예요.
#Person2#: 도와주셔서 감사합니다, 의사 선생님.
'''

messages = [
    {"role": "system", "content": f"{PROMPT}"},
    {"role": "user", "content": f"{instruction}"}
    ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=400,
    eos_token_id=terminators,
    do_sample=False,
    repetition_penalty = 1.1
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))